<a href="https://colab.research.google.com/github/GayathriShrikanth/J046NLP/blob/master/J046_NLP_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gayathri Shrikanth
J046
NLP M2


In [293]:
import nltk
import itertools
import pandas as pd
from gensim import models
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from string import digits,punctuation 
from nltk.tokenize import word_tokenize
from gensim.test.utils import get_tmpfile, common_texts
from bs4 import BeautifulSoup
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis
import os



nltk.download('punkt')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

remove_digits = str.maketrans('', '', digits)
stop_words = stopwords.words('german')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Reading the dataset

In [274]:
! wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true


--2020-03-03 16:41:36--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 140.82.118.4
Connecting to github.com (github.com)|140.82.118.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 16:41:37--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 16:41:37--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.13

In [275]:
# Unzipping the Datasets
! mkdir 10kGNAD
! unzip 10k-german-news-articles.zip?raw=true -d 10kGNAD

mkdir: cannot create directory ‘10kGNAD’: File exists
Archive:  10k-german-news-articles.zip?raw=true
replace 10kGNAD/Annotations.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [276]:
articles=pd.read_csv("10kGNAD/Articles.csv")
articles.head()

,ID_Article,Path,publishingDate,Title,Body
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro..."


In [0]:
#Function to remove html script
def remove_html(str):
  soup = BeautifulSoup(str)
  for script in soup(["script", "style"]):
      script.extract()    # rip it out
  text = soup.get_text()
  lines = (line.strip() for line in text.splitlines())
  block = (phrase.strip() for line in lines for phrase in line.split("  "))
  text = '\n'.join(block for block in block if block)
  return(text)

In [0]:
#Functions for preprocessing
def prepro(str):
  str = str.lower()
  str = str.translate(remove_digits)
  str = re.sub(r'[^\w\s]',' ',str)
  token = word_tokenize(str)
  words = [word for word in token if word not in stop_words and word not in punctuation]
  return words

def prepro2(str):
  str = str.lower()
  str = str.translate(remove_digits)
  str = re.sub(r'[^\w\s]',' ',str)
  return str

#Pipeline function
def pipeline(data):
  text=word_tokenize(data)
  tokenizer = MWETokenizer()
  textmw= tokenizer.tokenize(data.split())
  lemmatizer = WordNetLemmatizer()
  text3=lemmatizer.lemmatize(data)
  text4=nltk.pos_tag(data)
  words={"tokenizer":text,"lemmetizer":text3, "pos_tag":text4}
  return words

In [0]:
articles["text"]=articles.Body.apply(remove_html)
articles['Text_tok'] = articles.text.apply(prepro)

In [294]:
#Df considering only first 500 rows
df=articles[:1000]
df=pd.DataFrame(df)
df.head()

,ID_Article,Path,publishingDate,Title,Body,text,Text_tok
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",Abonnieren Sie einen unserer Newsletter und la...,"[abonnieren, unserer, newsletter, lassen, aktu..."
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",Werden Sie Teil von derStandard.at!10. Dezembe...,"[teil, derstandard, at, dezember, postingsmit,..."
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",Die Smartphone-App bietet Nachrichten in Echtz...,"[smartphone, app, bietet, nachrichten, echtzei..."
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",Wie sieht Ihr Alltag als Linkshänder aus? Erle...,"[sieht, alltag, linkshänder, erledigen, arbeit..."
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",Erzählen Sie uns von Ihren Erlebnissen rund um...,"[erzählen, erlebnissen, rund, weltpolitischen,..."


Main function to predict topics

In [0]:
#function to classify using LDA
def classify(df,x):
  def dummy_fun(doc):
      return doc
  vectorizer = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,token_pattern=None)
  vectors = vectorizer.fit_transform(df[x])
  feature_names = vectorizer.get_feature_names()
  dense = vectors.todense()
  denselist = dense.tolist()
  tfidf_df = pd.DataFrame(denselist, columns=feature_names)
  topics=[]
  number_topics = 5
  number_words = 5
  lda = LDA(n_components=number_topics, n_jobs=-1)
  lda.fit(tfidf_df)
  print("Topics found via LDA:")
  words = vectorizer.get_feature_names()
  for topic_idx, topic in enumerate(lda.components_):  
        topics.append(" ".join([words[i] for i in topic.argsort()[:-number_words - 1:-1]]))
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-number_words - 1:-1]]))
  prob_list=lda.transform(tfidf_df)
  top=[]
  for prob in prob_list:
    top.append(topics[np.argmax(prob)])
    #df["topic_pred"]=topic
  return top
  

In [296]:
p_top =classify(df, "Text_tok")
df["pred_topic"]=p_top


Topics found via LDA:

Topic #0:
mehr prozent wien sagte sei

Topic #1:
sei mehr österreich fpö zwei

Topic #2:
prozent eu mehr sei wurde

Topic #3:
wurde prozent mehr us sei

Topic #4:
prozent wurde fpö apple spö


In [297]:
df['Title_tok'] = df.text.apply(prepro)
t_top =classify(df, 'Title_tok')
df["pred_title"]=t_top


Topics found via LDA:

Topic #0:
prozent fpö spö mehr österreich

Topic #1:
mehr eu sei österreich wurde

Topic #2:
eu mehr euro apa sagte

Topic #3:
apple wurde zwei us euro

Topic #4:
apple worden agenturmeldungen basierenden volltext


In [298]:
df['pre_title'] = df.Title.apply(prepro)
df['pre_topic_tok'] = df.pred_topic.apply(prepro)
df['pre_title_tok'] = df.pred_title.apply(prepro)
df=df.drop(["Text_tok"], 1)
df.head()

,ID_Article,Path,publishingDate,Title,Body,text,pred_topic,Title_tok,pred_title,pre_title,pre_topic_tok,pre_title_tok
0,1,Newsroom/User/Community,2012-05-26 03:00:19.23,Die Newsletter von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",Abonnieren Sie einen unserer Newsletter und la...,mehr prozent wien sagte sei,"[abonnieren, unserer, newsletter, lassen, aktu...",mehr eu sei österreich wurde,"[newsletter, derstandard, at]","[mehr, prozent, wien, sagte, sei]","[mehr, eu, sei, österreich, wurde]"
1,2,Newsroom/User/Community/Regeln,2012-05-26 12:12:19.46,Werden Sie Teil von derStandard.at!,"<div class=""diashow"" id=""objectContent""><meta ...",Werden Sie Teil von derStandard.at!10. Dezembe...,mehr prozent wien sagte sei,"[teil, derstandard, at, dezember, postingsmit,...",prozent fpö spö mehr österreich,"[teil, derstandard, at]","[mehr, prozent, wien, sagte, sei]","[prozent, fpö, spö, mehr, österreich]"
2,3,Diverses/mobil,2013-11-22 12:15:00.00,Die Android App von derStandard.at,"<div class=""section"" id=""content-main"" itempro...",Die Smartphone-App bietet Nachrichten in Echtz...,prozent eu mehr sei wurde,"[smartphone, app, bietet, nachrichten, echtzei...",prozent fpö spö mehr österreich,"[android, app, derstandard, at]","[prozent, eu, mehr, sei, wurde]","[prozent, fpö, spö, mehr, österreich]"
3,4,Newsroom/User/mitmachen/Mitreden,2014-08-13 05:30:00.00,Welche Erfahrungen haben Sie als Linkshänder g...,"<div class=""section"" id=""content-main"" itempro...",Wie sieht Ihr Alltag als Linkshänder aus? Erle...,wurde prozent mehr us sei,"[sieht, alltag, linkshänder, erledigen, arbeit...",prozent fpö spö mehr österreich,"[erfahrungen, linkshänder, gemacht]","[wurde, prozent, mehr, us, sei]","[prozent, fpö, spö, mehr, österreich]"
4,5,Newsroom/User/mitmachen/Mitreden,2014-08-27 12:27:01.09,Wie haben Sie das Jahr 1989 erlebt?,"<div class=""section"" id=""content-main"" itempro...",Erzählen Sie uns von Ihren Erlebnissen rund um...,wurde prozent mehr us sei,"[erzählen, erlebnissen, rund, weltpolitischen,...",prozent fpö spö mehr österreich,"[jahr, erlebt]","[wurde, prozent, mehr, us, sei]","[prozent, fpö, spö, mehr, österreich]"


In [0]:
def l2s(plist):
  return str(' '.join(plist))
df['pre_title'] = df.pre_title.apply(l2s)

Finding similarity

In [0]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word')
actual_title_vectors = tfidf_vectorizer.fit_transform(df['pre_title'])
predicted_title_vectors = tfidf_vectorizer.transform(df["pred_topic"])
actual_title_list=actual_title_vectors.todense().tolist()
predicted_title_list=predicted_title_vectors.todense().tolist()


Similarity score

In [302]:
import scipy
cosine_sim = []
for i in range(0,1000):
  cosine_sim.append(scipy.spatial.distance.cosine(actual_title_list[i], predicted_title_list[i]))
print(cosine_sim)
avg=0
for j in cosine_sim:
  if j==1.0:
    avg+=j
print("Average similarity score is", avg/len(cosine_sim))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8085134665415266, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8509927829354389, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8962052227929205, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8735046261179467, 1.0, 0.8238872445547061, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8751969493620022, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8323773780526912, 0.7628231371864728, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7722783406070601, 1.0, 1.0,

pyLDAvis vizualization

In [0]:
#!pip install pyLDAvis
lda_display = sklearn_lda.prepare(lda,np.matrix(df3),vectorizer)
pyLDAvis.display(lda_display)        
#pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(number_topics) +'.html')

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
